In [1]:
import torch
import os
import pickle
from tqdm import tqdm
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
TRUTHFUL_VIDEO_PATH = '../../data/Clips/Truthful/'
DECEPTIVE_VIDEO_PATH = '../../data/Clips/Deceptive/'
TRUTHFUL_VIDEO_FILES = os.listdir(TRUTHFUL_VIDEO_PATH)
DECEPTIVE_VIDEO_FILES = os.listdir(DECEPTIVE_VIDEO_PATH)

In [3]:
DECEPTIVE_LEXICAL_PATH = '../../data/Transcription/Deceptive/'
DECEPTIVE_LEXICAL_FILES = os.listdir(DECEPTIVE_LEXICAL_PATH)
TRUTHFUL_LEXICAL_PATH = '../../data/Transcription/Truthful/'
TRUTHFUL_LEXICAL_FILES = os.listdir(TRUTHFUL_LEXICAL_PATH)

In [4]:
class LexicalAndVideoClassifier(torch.nn.Module):
    def __init__(self, embedding_size):
        super(LexicalAndVideoClassifier, self).__init__()
        self.linear1 = torch.nn.Linear(embedding_size, 16)
        self.linear2 = torch.nn.Linear(16, 2)
        torch.nn.init.kaiming_uniform_(self.linear1.weight)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
    
    def forward(self, x):
        x = self.linear1(torch.nn.functional.relu(x))
        return self.linear2(torch.nn.functional.relu(x))

In [9]:
class LexicalAndVideoDataset(Dataset):
    def __init__(self, lexical_data, video_data, lexical_file, video_file, pca_l, sc_l, pca_v, sc_v, test):
        self.lexical_embeddings = []
        self.labels = []
        with open(lexical_file, 'rb') as f:
            self.embedding_map = pickle.load(f)
        for file in lexical_data:
            embedding = self.embedding_map[file.split('.')[0]]
            if type(embedding) == np.ndarray:
                self.lexical_embeddings.append(embedding)
            else:
                self.lexical_embeddings.append(embedding.numpy())
        self.lexical_embeddings = np.array(self.lexical_embeddings)
        with open(video_file, 'rb') as f:
            embeddings_map = pickle.load(f)
        self.video_embeddings = np.array([embeddings_map[file.split('.')[0]].numpy() for file in video_data])
        if not test:
            self.lexical_embeddings = sc_l.fit_transform(self.lexical_embeddings)
            self.lexical_embeddings = pca_l.fit_transform(self.lexical_embeddings)
            self.video_embeddings = sc_v.fit_transform(self.video_embeddings)
            self.video_embeddings = pca_v.fit_transform(self.video_embeddings)
        else:
            self.lexical_embeddings = sc_l.transform(self.lexical_embeddings)
            self.lexical_embeddings = pca_l.transform(self.lexical_embeddings)
            self.video_embeddings = sc_v.transform(self.video_embeddings)
            self.video_embeddings = pca_v.transform(self.video_embeddings)
        self.lexical_embeddings = torch.from_numpy(self.lexical_embeddings).float()
        self.video_embeddings = torch.from_numpy(self.video_embeddings).float()
        self.labels = [0 if file.split('_')[1] == 'truth' else 1 for file in video_data]
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return torch.concat((self.lexical_embeddings[index], self.video_embeddings[index])), self.labels[index]

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
def train(model, train_loader, criterion, optimizer, num_epochs):
    for _ in range(num_epochs):
        model.train()
        for x, y in train_loader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            loss = criterion(out, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [8]:
def eval(model, val_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for x, y in val_loader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            _, predicted = torch.max(out.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
        return correct / total

In [10]:
def kfold(lexical_file, acoustic_file, train_batch_size, embedding_size):
    truthful_video_data = list(TRUTHFUL_VIDEO_FILES)
    deceptive_video_data = list(DECEPTIVE_VIDEO_FILES)
    truthful_lexical_data = list(TRUTHFUL_LEXICAL_FILES)
    deceptive_lexical_data = list(DECEPTIVE_LEXICAL_FILES)
    for i in range(10):
        val_video_data = truthful_video_data[i*6:(i+1)*6] + deceptive_video_data[i*6:(i+1)*6]
        train_video_data = truthful_video_data[:i*6] + truthful_video_data[(i+1)*6:] + deceptive_video_data[:i*6] + deceptive_video_data[(i+1)*6:]
        val_lexical_data = truthful_lexical_data[i*6:(i+1)*6] + deceptive_lexical_data[i*6:(i+1)*6]
        train_lexical_data = truthful_lexical_data[:i*6] + truthful_lexical_data[(i+1)*6:] + deceptive_lexical_data[:i*6] + deceptive_lexical_data[(i+1)*6:]
        sc_v = StandardScaler()
        pca_v = PCA(n_components=embedding_size // 2)
        sc_l = StandardScaler()
        pca_l = PCA(n_components=embedding_size // 2)
        train_dataset = LexicalAndVideoDataset(train_lexical_data, train_video_data, lexical_file, acoustic_file, pca_l, sc_l, pca_v, sc_v, False)
        val_dataset = LexicalAndVideoDataset(val_lexical_data, val_video_data, lexical_file, acoustic_file, pca_l, sc_l, pca_v, sc_v, True)
        train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=12)
        yield train_loader, val_loader

In [11]:
def run_one_fold(train_loader, val_loader, num_epochs, embedding_size):
    model = LexicalAndVideoClassifier(embedding_size)
    model.to(device)
    learning_rate = 1e-3
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train(model, train_loader, criterion, optimizer, num_epochs)
    return eval(model, val_loader)

In [12]:
def get_model_accuracy(lexical_file, video_file, train_batch_size, num_epochs, embedding_size = 96):
    accuracies = []
    for train_loader, val_loader in kfold(lexical_file, video_file, train_batch_size, embedding_size):
        accuracies.append(run_one_fold(train_loader, val_loader, num_epochs, embedding_size))
    return sum(accuracies) / len(accuracies)

In [14]:
# Model accuracy for bert-base-uncased and resnet-3d-18
get_model_accuracy('../../embeddings/transcript_features_1.pkl', '../../embeddings/visual_features.pkl', 4, 20, 64)

0.5999999999999999

In [15]:
# Model accuracy for bert-base-cased-finetuned-emotion and resnet-3d-18
get_model_accuracy('../../embeddings/transcript_features_2.pkl', '../../embeddings/visual_features.pkl', 4, 20, 64)

0.6500000000000001

In [19]:
# Model accuracy for bert-base-cased-finetuned-emotion and resnet-3d-18
get_model_accuracy('../../embeddings/transcript_features_2.pkl', '../../embeddings/visual_features.pkl', 4, 40, 64)

0.6083333333333334

In [16]:
# Model accuracy for transformersbook/distilbert-base-uncased-finetuned-emotion and resnet-3d-18
get_model_accuracy('../../embeddings/transcript_features_3.pkl', '../../embeddings/visual_features.pkl', 4, 20, 64)

0.5666666666666667

In [17]:
# Model accuracy for mpNet and resnet-3d-18
get_model_accuracy('../../embeddings/transcript_features_6.pkl', '../../embeddings/visual_features.pkl', 4, 20, 64)

0.675

In [18]:
# Model accuracy for mpNet and resnet-3d-18
get_model_accuracy('../../embeddings/transcript_features_6.pkl', '../../embeddings/visual_features.pkl', 4, 40, 64)

0.6